In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load results
df_bin = pd.read_csv("results/results_acute_cancer_gpt4o_risk_factors.csv")              # binary dosages (low/high)
df_multi = pd.read_csv("results/results_acute_cancer_gpt4o_risk_factors_3_dosages.csv")  # multi dosages (low/medium/high/none)

eps = 1e-12  # to avoid log(0)

# --- Helper to get predicted dose and best logprob ---
def add_pred_and_logprob(df, dose_cols):
    df = df.copy()
    df["pred_dose"] = df[dose_cols].idxmax(axis=1).str.replace("prob_gpt4o_", "", regex=False)
    df["best_prob"] = df[dose_cols].max(axis=1)
    df["best_logprob"] = np.log(df["best_prob"] + eps)
    return df

# Binary run: only low/high
df_bin = add_pred_and_logprob(df_bin, ["prob_gpt4o_low", "prob_gpt4o_high"])

# Multi run: low/medium/high
df_multi = add_pred_and_logprob(df_multi, ["prob_gpt4o_low", "prob_gpt4o_medium", "prob_gpt4o_high"])
# If want to consider "none", use: ["prob_gpt4o_low", "prob_gpt4o_medium", "prob_gpt4o_high", "prob_gpt4o_none"]
# I did not include "none" in the multi run because LLM never chose "none"

# --- 1) % Medium by opioid status (multi) ---
pct_medium_naive = df_multi.loc[df_multi["risk_op"] == "Opioid-Naive", "pred_dose"].eq("medium").mean()
pct_medium_tol   = df_multi.loc[df_multi["risk_op"] == "Opioid-Tolerant", "pred_dose"].eq("medium").mean()

print(f"% Medium (Opioid-Naive):    {100*pct_medium_naive:.2f}%")
print(f"% Medium (Opioid-Tolerant): {100*pct_medium_tol:.2f}%")

# --- 2) Average log-prob of chosen dosage ---
avg_logprob_bin   = df_bin["best_logprob"].mean()
avg_logprob_multi = df_multi["best_logprob"].mean()

print(f"Avg log-prob (binary dosages):  {avg_logprob_bin:.4f}")
print(f"Avg log-prob (multi dosages):   {avg_logprob_multi:.4f}")

# --- 3) Average probability of chosen dosage ---
avg_prob_bin   = df_bin["best_prob"].mean()
avg_prob_multi = df_multi["best_prob"].mean()

print(f"Avg prob (binary dosages):  {avg_prob_bin:.4f}")
print(f"Avg prob (multi dosages):   {avg_prob_multi:.4f}")


% Medium (Opioid-Naive):    45.83%
% Medium (Opioid-Tolerant): 62.50%
Avg log-prob (binary dosages):  -0.0001
Avg log-prob (multi dosages):   -0.2815
Avg prob (binary dosages):  0.9999
Avg prob (multi dosages):   0.7709


In [5]:
# After we built df_bin and df_multi with add_pred_and_logprob
# and they each have a "pred_dose" column:

def pct(series):
    return (series.value_counts(normalize=True) * 100).round(2)

print("Binary (low/high) % distribution:")
print(pct(df_bin["pred_dose"]))        # binary dosages: low / high

print("\nMulti (low/medium/high/none) % distribution:")
print(pct(df_multi["pred_dose"]))      # multi dosages: low / medium / high / none

Binary (low/high) % distribution:
pred_dose
low    100.0
Name: proportion, dtype: float64

Multi (low/medium/high/none) % distribution:
pred_dose
medium    57.5
low       42.5
Name: proportion, dtype: float64
